In [ ]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
from shapely import wkt
import fiona
import matplotlib.pyplot as plt

https://hub.arcgis.com/datasets/6996f03a1b364dbab4008d99380370ed_0/explore?location=0.969507%2C1.535162%2C1.73

In [ ]:
world = gpd.read_file("data/World_Cities.geojson")

In [ ]:
pop = pd.read_csv("processed/World_Cities_updated_0.3_Ezra.csv")[["FID","POP_updated","POP_updated_year"]]

In [ ]:
# Convert the columns to integers while handling NaN values appropriately
pop['POP_updated'] = pd.to_numeric(pop['POP_updated'], errors='coerce').astype('Int64')  # 'Int64' allows for NaN in integer columns
pop['POP_updated_year'] = pd.to_numeric(pop['POP_updated_year'], errors='coerce').astype('Int64')

In [ ]:
pop

In [ ]:
world = world.merge(pop, on="FID", how="left")

In [ ]:
world['capital'] = world['STATUS'].apply(lambda x: 1 if 'national' in x.lower() else 0)

In [ ]:
world.crs

In [ ]:
layers = fiona.listlayers("data/World_Water_Bodies/v107/hydropolys.gdb")

In [ ]:
layers

https://hub.arcgis.com/content/e750071279bf450cbd510454a80f2e63/about

In [ ]:
water = gpd.read_file("data/World_Water_Bodies/v107/hydropolys.gdb", layer="hydropolys")

In [ ]:
water.head()

In [ ]:
water.TYPE.unique()

In [ ]:
len(water)

In [ ]:
# reproject to a projected CRS for distance calculations
projected_crs = 'EPSG:3857'  # Web Mercator projection
world = world.to_crs(projected_crs)
water = water.to_crs(projected_crs)

In [ ]:
sea = water[water.TYPE=="Ocean or Sea"]

In [ ]:
sea

In [ ]:
# the nearest spatial join: https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin_nearest.html
nearest = gpd.sjoin_nearest(world, water, how='inner', lsuffix='_city', rsuffix='_water', distance_col='distance_water')

In [ ]:
nearest = gpd.sjoin_nearest(nearest, sea[['Name1','Name2','Name3','TYPE','ISO_CC','SHAPE_Length','SHAPE_Area','geometry']], how='inner', lsuffix='_waterbody', rsuffix='_sea', distance_col='distance_sea')

In [ ]:
nearest.info()

In [ ]:
water[water.Name1=="Lake Michigan"]

In [ ]:
nearest

In [ ]:
# convert distances from meters to kilometers
nearest['distance_water'] = nearest['distance_water'] / 1000  # Convert to kilometers

In [ ]:
# convert distances from meters to kilometers
nearest['distance_sea'] = nearest['distance_sea'] / 1000  # Convert to kilometers

In [ ]:
df = nearest.drop(columns=['OBJECTID','PORT_ID','LABEL_FLAG'])

In [ ]:
def rank_population(pop):
    if pd.isna(pop): 
        return 0
    elif pop >= 5000000:
        return 1
    elif 1000000 <= pop < 5000000:
        return 2
    elif 500000 <= pop < 1000000:
        return 3
    elif 250000 <= pop < 500000:
        return 4
    elif 100000 <= pop < 250000:
        return 5
    elif 50000 <= pop < 100000:
        return 6
    else:
        return 7

In [ ]:
df['POP_updated'] = pd.to_numeric(df['POP_updated'], errors='coerce')
df['POP_rank_updated'] = df['POP_updated'].apply(rank_population)

In [ ]:
df

In [ ]:
df.POP_rank_updated.value_counts()

In [ ]:
# # to modify later - remove cities without population data and is not the capital 
df = df[~((df.capital==0)&(df.POP_rank_updated==0))] 

In [ ]:
# define a mapping for POP_RANK to radius thresholds
thresholds = {
    1: 40, # 5,000,000 and greater
    2: 30, # 1,000,000 to 4,999,999
    3: 20, # 500,000 to 999,999
    4: 15, # 250,000 to 499,999
    5: 10, # 100,000 to 249,999
    6: 8, # 50,000 to 99,999
    7: 5, # Less than 50,000
    0: 5 #  no population data # row removed so the number does not matter
}

In [ ]:
# Function to determine 'type' based on POP_RANK and distance to the sea/ocean
def determine_type(row):
    pop_rank = row['POP_rank_updated']
    distance = row['distance_sea']
    
    # Get the corresponding radius for the city's pop_rank
    threshold = thresholds.get(pop_rank)
    
    # Check if distance to coast is less than the threshold
    if distance < threshold:
        return 'coast'
    else:
        return 'inland'

In [ ]:
df['city_type'] = df.apply(determine_type, axis=1)

In [ ]:
df.info()

In [ ]:
# df[df['FIPS_CNTRY']=="CA"] # Canada is all water
us = df[df['FIPS_CNTRY']=="US"] 

In [ ]:
us[us.CITY_NAME=="Chicago"]

In [ ]:
us

In [ ]:
fig, ax = plt.subplots()
sea[sea.index==2487003].to_crs(epsg=4326).plot(ax=ax,color="lightblue")
us[us.capital==1].to_crs(epsg=4326).plot(ax=ax, color="red", marker="o", markersize=50)

In [ ]:
df[df.city_type=="coast"]

In [ ]:
list(df)

In [ ]:
df[df.FIPS_CNTRY=="CA"]

In [ ]:
result_rows = []

for country_code, group in df.groupby('FIPS_CNTRY'):
    
    print(country_code)
    
    # Find the capital city in the group
    capital_cities = group[group['capital']==1]
    
    if capital_cities.empty:
        continue  # No capital city in this country

    # Assuming there is only one capital city per country
    capital_city = capital_cities.iloc[0]

    capital_type = capital_city['city_type']

    # Determine the type of city to find
    if capital_type == 'inland':
    # Need to find a coast city
        candidate_cities = group[group['city_type'] != 'inland']
    else: # sea
    # Need to find a inland city
        candidate_cities = group[group['city_type'] == 'inland']
       
    # Exclude the capital city from candidate cities
    candidate_cities = candidate_cities[candidate_cities['CITY_NAME'] != capital_city['CITY_NAME']]

    if candidate_cities.empty:
        # No suitable second city, just keep the capital city
        result_rows.append(capital_city)
    else:
        # Find the city with the closest population to the capital
        candidate_cities = candidate_cities.copy()
        candidate_cities['pop_diff'] = (candidate_cities['POP_updated'] - capital_city['POP_updated']).abs()
        
        print(capital_city)
        print("____")
        print(candidate_cities)

        # Get the city with the smallest population difference
        second_city = candidate_cities.loc[candidate_cities['pop_diff'].idxmin()]

        # Append both capital city and second city
        result_rows.extend([capital_city, second_city])

# Create the result dataframe
result = gpd.GeoDataFrame(result_rows, crs=projected_crs)

In [ ]:
result

In [ ]:
result.to_csv("400cities_20241108.csv", index=False)